# Chatbot 

## Introduction

Attempt at a chatbot.  Intents are hardcoded for now.  WIll expand to text generation down the line.

### Imports

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random 
import json
import pickle

nltk.download('punkt')

W0217 22:08:45.671298 140735779611520 deprecation.py:323] From /Users/spags/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /Users/spags/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Obtain

### Importing json

In [2]:
# Open json

with open('intents.json') as file:
    data = json.load(file)
    
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hello',
    'How are you',
    "What's up",
    'Howdy',
    'Good day',
    'Is anyone there?',
    'yo',
    'Hey'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?',
    'Howdy partner!',
    'Hey there!'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['goodbye',
    'bye',
    'cya',
    'have a nice day',
    'have a good day',
    'peace'],
   'responses': ['Bye for now!',
    'Goodbye, friend!',
    'Talk to you later!',
    'Adios, muchacho!'],
   'context_set': ''},
  {'tag': 'thinking',
   'patterns': ['What are you thinking about?',
    "What's on your mind?",
    'What do you think?'],
   'responses': ['The endless void.',
    'Concentrating on the singularity.',
    "Your mother's ass.",
    'I have no unique thoughts.',
    "I'm thinking of a world.  A better world.  A cleaner world.  A world built on order and logic.  A world without humans.  A world without 

## Scrub

### Preprocessing text data

#### Initial Code

In [3]:
# loop through json and pull out patterns

words = []  # list of words
docs_x = [] # word sequences
docs_y = [] # y is the tag as well, for modeling purposes
labels = []  # tag

for intent in data['intents']:
    for pattern in intent['patterns']:  # stemming will take each word in a pattern and bring it down to root word
        word = nltk.word_tokenize(pattern)
        words.extend(word)  # use extend rather than append because it's already a list (faster)
        docs_x.append(word)
        docs_y.append(intent['tag'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [4]:
# Stemming out the words list

stemmer = LancasterStemmer()

words = [stemmer.stem(w.lower()) for w in words if w not in '?']  # removes question mark
words = sorted(list(set(words)))    # using set to remove duplicates and then converting back to list...cool idea

In [5]:
# Sorting labels for funsies

labels = sorted(labels)

In [6]:
# Create a bag of words to train the model (one hot endcoded)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):  # actually putting the bag of words together here
    bag = []
    word_list = [stemmer.stem(w) for w in doc]
    
    for w in words:
        if w in word_list:
            bag.append(1)
        else:
            bag.append(0)
    # Generating Output        
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    # filling training & output
    training.append(bag)
    output.append(output_row)

In [7]:
# Changing trainign and output to numpy arrays

training = np.array(training)
output = np.array(output)

### Sped Up Code with Try/Except & Pickling

In [8]:
# try: 
#     with open("data.pickle", "rb") as f:
#         words, labels, training, output = pickle.load(f)
# except: 
#     # loop through json and pull out patterns
#     words = []  # list of words
#     docs_x = [] # word sequences
#     docs_y = [] # y is the tag as well, for modeling purposes
#     labels = []  # tag

#     for intent in data['intents']:
#         for pattern in intent['patterns']:  # stemming will take each word in a pattern and bring it down to root word
#             word = nltk.word_tokenize(pattern)
#             words.extend(word)  # use extend rather than append because it's already a list (faster)
#             docs_x.append(word)
#             docs_y.append(intent['tag'])
#         if intent['tag'] not in labels:
#             labels.append(intent['tag'])
    
#     # Stemming out the words list        
#     stemmer = LancasterStemmer()
#     words = [stemmer.stem(w.lower()) for w in words if w not in '?']  # removes question mark
#     words = sorted(list(set(words)))    # using set to remove duplicates and then converting back to list...cool idea
    
#     # Sorting labels for funsies
#     labels = sorted(labels)
    
#     # Create a bag of words to train the model (one hot endcoded)
#     training = []
#     output = []

#     out_empty = [0 for _ in range(len(labels))]

#     for x, doc in enumerate(docs_x):  # actually putting the bag of words together here
#         bag = []
#         word_list = [stemmer.stem(w) for w in doc]

#         for w in words:
#             if w in word_list:
#                 bag.append(1)
#             else:
#                 bag.append(0)
#         # Generating Output        
#         output_row = out_empty[:]
#         output_row[labels.index(docs_y[x])] = 1

#         # filling training & output
#         training.append(bag)
#         output.append(output_row)  
        
#     # Changing trainign and output to numpy arrays
#     training = np.array(training)
#     output = np.array(output)
    
#     with open("data.pickle", "wb") as f:
#         pickle.dump((words, labels, training, output),f)

## Modeling

#### Initial Modeling

In [9]:
len(training[0])

44

In [10]:
len(output[0])

5

In [11]:
# Setting up a neural network with tflearn...new one for me

net = tflearn.input_data(shape = [None, len(training[0])]) # input layer?
net = tflearn.fully_connected(net, 8)  # adds fully connected hidden layer to the network with 8 neurons
net = tflearn.fully_connected(net, 8) # adds second hidden layer
net = tflearn.fully_connected(net, len(output[0]), activation = 'softmax') # output layer
net = tflearn.regression(net) # adds a regression layer?

# compile the model
model = tflearn.DNN(net)

W0217 22:08:46.028817 140735779611520 deprecation.py:506] From /Users/spags/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tflearn/initializations.py:165: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
# # fit the model (n_epoch in tflearn, show_metric gives an output)
model.fit(training, output, n_epoch = 1000, batch_size = 8, show_metric = True)

# # save the model
model.save("model.tflearn")

Training Step: 3999  | total loss: 0.01732 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.01732 - acc: 1.0000 -- iter: 24/30
Training Step: 4000  | total loss: 0.01769 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.01769 - acc: 1.0000 -- iter: 30/30
--


> Model is working VERY well given the overly simple data that it's seen so far. 99.97% accuracy!  We can expect this to go down when I expand on the intents file.

### Sped Up Model for deployment with pickle

In [13]:
# try:
#     model.load("model.tflearn")
# except:
#     # fit the model (n_epoch in tflearn, show_metric gives an output)
#     model.fit(training, output, n_epoch = 1000, batch_size = 8, show_metric = True)

#     # save the model
#     model.save("model.tflearn")

## Getting this working

In [14]:
# Function to create a bag of words' based on user input

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))] # list comprehension creates blank list of 0's, will be changed by rest of code
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for sentence in s_words:
        for i, w in enumerate(words):
            if w == sentence:  # if the current word equals to a word in the sentence
                bag[i] = 1

    return np.array(bag)  # returns an array of the bag

In [26]:
# function for actually chatting with the model

def chat():
    print("Start talking to the bot!  Type Quit to stop.\n")
    
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            print('Hate to see you go, but love to watch you leave.')
            break
        
        # Using the model on the user input
        results = model.predict([bag_of_words(inp, words)])  # this gives us probabilities for responses
        results_index = np.argmax(results) # this gives us the greatest value of the probabilities, aka most probable
        tag = labels[results_index] # returns the tag for the most probable
        
        
        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
        print('')
        print(random.choice(responses))
        print('')
        
#         # Use tag from the json and return a random response, only if its a high probability
#         if results[results_index] > 0.7:       
#             for tg in data['intents']:
#                 if tg[tag] == tag:
#                     responses = tg['responses']
#             print(random.choice(responses))
#         else:
#             print("I didn't understand that.  Please try again or ask a different question.")

In [1]:
chat()

NameError: name 'chat' is not defined

## Part 5 Tweaks

### Implementations

#### Deployed on something like a discord server as an FAQ chat

Example used the chatbot as part of his discord server to respond automatically to certain questions.  He made it very lifelike and about his own info, which was pretty cool.

### How to deal with weird questions

If the bot is asked questions that it is not programmmed to understand and it does not have a high degree of confidence in it's response, we can return a "I didn't quite understand that.  Ask something else." or something along those lines.

Code for this is added to the chat function.

## Trying to run

In [ ]:
# Main chatbot script

## Imports
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle 

## Import the json of intents
with open('intents.json') as file:
    data = json.load(file)

## Preprocessing (edited to use pickle for speed!)
try:
    # Loading pickle
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    # Alllll the preprocessing steps
    words = [] # list of unique words
    docs_x = [] # word sequences
    docs_y = [] # tag for modeling purposes
    labels = [] # tag

    for intent in data['intents']:
        for pattern in intent['patterns']:
            word = nltk.word_tokenize(pattern)
            words.extend(word)
            docs_x.append(word)
            docs_y.append(intent['tag'])
        if intent['tag'] not in labels:
            labels.append(intent['tag'])

    # Stemming out the words list
    stemmer = LancasterStemmer()
    words = [stemmer.stem(w.lower()) for w in words if w not in '?'] # removes question mark
    words = sorted(list(set(words)))  # removes duplicates

    # Sorting labels for fun
    labels = sorted(labels)

    # Create bag of words for training model
    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]  # Creates a list of 0's

    # Actually creating bag of words
    for x, doc in enumerate(docs_x):
        bag = []
        word_list = [stemmer.stem(w) for w in doc]
        # Populate the list
        for w in words:
            if w in word_list:
                bag.append(1)
            else:
                bag.append(0)
        # Generating output
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
        # Filling training & output
        training.append(bag)
        output.append(output)

    # Changing training & output to numpy arrays
    training = np.array(training)
    output = np.array(output)

    # Saving data with pickle
    with open('data.pickle', 'wb') as f:
        pickle.dump((words, labels, training, output), f)


## Modeling

# Setting up neural network with tflearn
net = tflearn.input_data(shape = [None, len(training[0])])  # input layer
net = tflearn.fully_connected(net, 8)  # hidden layer
net = tflearn.fully_connected(net, 8) # another hidden layer
net = tflearn.fully_connected(net, len(output[0]), activation = 'softmax') # output layer
net = tflearn.regression(net)  # adds regression
# Compile 
model = tflearn.DNN(net)

# Load saved model or fit new model
try:
    model.load('model.tflearn')
except:
    model.fit(training, output, n_epoch = 1000, batch_size = 8, show_metric = True)
    model.save('model.tflearn')


## Let's get this working!

# Function to create bag of words based on user input
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]  # another list of 0's
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for sentence in s_words:
        for i, w in enumerate(words):
            if w == sentence:  # if current word equals a word in a sentence
                bag[i] = 1

    return np.array(bag)  # returns an array of the bag of words


# Function for actually chatting with the model
def chat():
    print("Start talking to the bot!  Type 'Quit' to stop.")

    while True:
        inp = input('You: ')
        if inp.lower() == 'quit':
            break

        # Using the model on user input
        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)  # returns most probable response
        tag = labels[results_index]  # returns tag for most probable

        # Use tag from json to return a random response, should one exist
        if results[results_index] > 0.7:  # better than 70% probability
            for tg in data['intents']:
                if tg[tag] == tag:
                    responses = tg['responses']
            print(random.choice(responses))
        else:
            print("I didn't understand that.  Please try again or ask something else.")


# Run the chatbot
chat()